- google drive
  - "C:\APN\DATA\GoogleDrive\My Affirmations\affirmationBackup.zip"

- apk (no use)
  - Package [com.ascent.affirmations.myaffirmations] (7fa3df6):
  - path: /data/app/~~gDMvcg3lHuJP4JHcWh8b0A==/com.ascent.affirmations.myaffirmations-aGQtuh7Kps40L_Q50kCWYA==/base.apk


Python code:
1. 給定一個zip檔案: ex. path/affirmationBackup.zip
2. 將 path/affirmationBackup.zip 複製到工作目錄, 工作目錄設定成: 當下目錄下的 ./out_Myaffermation/230709 加上日期代碼
3. 將工作目錄的 path/affirmationBackup.zip 揭開壓縮
4. 編輯(刪除record)後，重新壓縮
5. 手動蓋過google drive上的affirmationBackup.zip
6. 手機上restore後，需要跳出程式，再進入，更動才會生效 （不確定)

失敗==>自己包的zip, restore似乎會失敗

In [19]:
#!/usr/bin/python
# -*- coding: utf-8 -*-
from __future__ import print_function
import os
import shutil
import zipfile
import sqlite3
import subprocess
from datetime import datetime


### [Functions]

In [20]:
"""
========================================================
Functions
========================================================
"""
def afftbl_show_all(db_file_path):
    # 连接到数据库
    conn = sqlite3.connect(db_file_path)
    cursor = conn.cursor()

    # 执行查询操作
    query = "SELECT * FROM AFFIRMATION;"
    cursor.execute(query)

    # 获取所有记录
    records = cursor.fetchall()

    # 输出记录
    for record in records:
        print(record)

    # 关闭连接
    conn.close()

def afftbl_find(db_file_path, search_string):
    # 连接到数据库
    conn = sqlite3.connect(db_file_path)
    cursor = conn.cursor()

    # 搜索特定字符串并获取匹配记录的索引
    query = f"SELECT * FROM AFFIRMATION WHERE affirmation LIKE '%{search_string}%';"
    cursor.execute(query)
    matching_rows = cursor.fetchall()

    for row in matching_rows:
        print(row)

    conn.close()
    match_row_ids = [row[0] for row in matching_rows]
    return match_row_ids


def afftbl_delete(db_file_path, row_ids):
    # 连接到数据库
    conn = sqlite3.connect(db_file_path)
    cursor = conn.cursor()

    # 删除匹配的记录
    for row in row_ids:
        print(f"delete row id: {row}")
        delete_query = f"DELETE FROM AFFIRMATION WHERE rowid = {row};"
        cursor.execute(delete_query)

    # 提交更改并关闭连接
    conn.commit()
    conn.close()


In [21]:
def compress_to_zip(source_dir, zip_path):
    # 压缩文件夹为zip，仅包含当前层级的目录名称
    current_dir = os.path.basename(source_dir)
    with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zip_ref:
        for root, dirs, files in os.walk(source_dir):
            for file in files:
                file_path = os.path.join(root, file)
                rel_path = os.path.relpath(file_path, source_dir)
                arcname = os.path.join(current_dir, rel_path)
                zip_ref.write(file_path, arcname=arcname)

def open_directory(directory):
    # 使用explorer打开目录
    subprocess.run(["explorer", directory])



### [Config]

In [25]:
"""
-----------------------------------------------
Config Setting
-----------------------------------------------
"""
# src_zip_file = r"C:\APN\DATA\GoogleDrive\My Affirmations\affirmationBackup.zip"
src_zip_file = r"C:\APN\DATA\GoogleDrive\affirmationBackup.zip"
_current_datetime = datetime.now().strftime("%Y%m%d_%H%M")
work_directory = os.path.join(os.getcwd(), "out-Myaffermations", _current_datetime)
_fname, _fext = os.path.splitext(os.path.basename(src_zip_file))
dest_zip_file = os.path.join(work_directory, _fname + "-org" + _fext)
db_file_path = os.path.join(work_directory, "My Affirmations","backup","affirmationDB.mab")
target_zip_file = os.path.join(work_directory, _fname + _fext)


print(f"src_zip_file    = '{src_zip_file}'")
print(f"work_directory  = '{work_directory}'")
print(f"dest_zip_file   = '{dest_zip_file}'")
print(f"db_file_path    = '{db_file_path}'")
print(f"target_zip_file = '{target_zip_file}'")

src_zip_file    = 'C:\APN\DATA\GoogleDrive\affirmationBackup.zip'
work_directory  = 'c:\APN\DATA\OneDrive\BIN\cls-toolbox\jupyterNotebook\out-Myaffermations\20230709_1746'
dest_zip_file   = 'c:\APN\DATA\OneDrive\BIN\cls-toolbox\jupyterNotebook\out-Myaffermations\20230709_1746\affirmationBackup-org.zip'
db_file_path    = 'c:\APN\DATA\OneDrive\BIN\cls-toolbox\jupyterNotebook\out-Myaffermations\20230709_1746\My Affirmations\backup\affirmationDB.mab'
target_zip_file = 'c:\APN\DATA\OneDrive\BIN\cls-toolbox\jupyterNotebook\out-Myaffermations\20230709_1746\affirmationBackup.zip'


### [Execute]

In [26]:
"""
-----------------------------------------------
uncompress
-----------------------------------------------
"""
os.makedirs(work_directory, exist_ok=True)
shutil.copyfile(src_zip_file, dest_zip_file)
with zipfile.ZipFile(dest_zip_file, "r") as zip_ref:
    zip_ref.extractall(work_directory)

print(f"uncompress: at f'{work_directory}'")

uncompress: at f'c:\APN\DATA\OneDrive\BIN\cls-toolbox\jupyterNotebook\out-Myaffermations\20230709_1746'


In [28]:
#...................................
# Step 1: Find String
#...................................
search_string = input("請輸入搜尋字串:")
print(f"search_string: {search_string}")
match_row_ids = afftbl_find(db_file_path, search_string)
print(f"match_row_ids = {match_row_ids}")

#...................................
# Step 2: Select to delete
#...................................
print("請輸入欲刪除的id(用逗號隔開,Enter表示输入完成):", flush=True)
todel_row_ids = input("請輸入欲刪除的id(用逗號隔開,Enter表示输入完成):")
print(f"todel_row_ids: {todel_row_ids}", flush=True) #debug
if todel_row_ids:
    todel_row_ids = [ int(x) for x in todel_row_ids.split(",")]
    todel_row_ids = [ x for x in todel_row_ids if x in match_row_ids]
print(f"todel_row_ids: {todel_row_ids}", flush=True)

#...................................
# Step 3: Delete
#...................................
afftbl_delete(db_file_path, todel_row_ids)
print("Table after delete:")
afftbl_find(db_file_path, search_string)

#...................................
# Step 4: compress
#...................................
compress_to_zip(os.path.join(work_directory,"My Affirmations"), target_zip_file)
open_directory(work_directory)
open_directory(os.path.dirname(src_zip_file))
print(f"Final Zip Done: {target_zip_file}")
print("Please copy to Google Drive and Restore!")


search_string: 社交
(244, '書摘', '社交媒體應該是一個平台，幫助你看到所愛之人的生活，而不是讓你去嫉妒別人。讓我們與喜樂的人同樂。讓我們培養感恩的心。', 1, None, 244, 17, '', '', None)
(245, '書摘', '你真正應該害怕的是錯過你身邊的人。當你緊盯著手機或平板電腦的時候，這就是你正在錯過的。你可能會錯過你孩子的成長過程。你可能會錯過享受一段親密的婚姻。你可能會錯過充滿意義的深厚友誼。你老是害怕錯過（社交媒體上的）某件事，結果可能會讓你錯過真正最重要的事。', 1, None, 245, 18, '', '', None)
(843, '處事原則', '社交高手聊天術：\r\n1. 別說「我也是」。 這種「我也是」的回答，是在告訴對方「這件事情沒什麼特別的」，，要問好玩嗎?\r\n2. 別只單純地回答別人的問題, 回答沒有的同時, 延續問對方: 那好玩嗎？\r\n3. 將對方說的話延展成新話題, 不要只說是噢，要說，我聽說那裏人都很好是真的嗎？延續新的話題\r\n4.  不要問是非題，要問申論題。不要問你一定很高興吧？要問那你是什麼心情？\r\n5.  順着對方問題提出反問，不要回答我們也是，可以反問那你都會怎麽解決？', 1, None, 843, 843, '', '', None)
match_row_ids = [244, 245, 843]
請輸入欲刪除的id(用逗號隔開,Enter表示输入完成):
todel_row_ids: 
todel_row_ids: 
Table after delete:
(244, '書摘', '社交媒體應該是一個平台，幫助你看到所愛之人的生活，而不是讓你去嫉妒別人。讓我們與喜樂的人同樂。讓我們培養感恩的心。', 1, None, 244, 17, '', '', None)
(245, '書摘', '你真正應該害怕的是錯過你身邊的人。當你緊盯著手機或平板電腦的時候，這就是你正在錯過的。你可能會錯過你孩子的成長過程。你可能會錯過享受一段親密的婚姻。你可能會錯過充滿意義的深厚友誼。你老是害怕錯過（社交媒體上的）某件事，結果可能會讓你錯過真正最重要的事。', 1, None, 245, 18, '', '', None)
(843, '處事原則', '社交高手聊天術